**Assignment set 3** by [Pegah Khazaei](https://github.com/pegahkhazaeii/Deep_Learning_F20_Assignments), October 10, 2020



In [ ]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets

In [ ]:
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
df = datasets.FashionMNIST('data/', download=True, transform=transforms.ToTensor())

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw


Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [ ]:
train_dataset, test_dataset = torch.utils.data.dataset.random_split(df, [42000 ,18000])

In [ ]:
# Hyper parameters
num_epochs = 5
num_classes = 10
batch_size = 100
learning_rate = 0.001

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)

In [ ]:
# Define the network architecture
from torch import nn, optim
import torch.nn.functional as F

model = nn.Sequential(nn.Linear(784, 256),
                      nn.ReLU(),

                      
                      nn.Linear(256, 128),
                      nn.Softmax(),

                      
                      nn.Linear(128, 64),
                      nn.Sigmoid(),

                      
                      nn.Linear(64, 32),
                      nn.ReLU(),

                      nn.Linear(32, 10),
                      nn.LogSoftmax(dim = 1)


                     )

                
print(model)

Sequential(
  (0): Linear(in_features=784, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=128, bias=True)
  (3): Softmax(dim=None)
  (4): Linear(in_features=128, out_features=64, bias=True)
  (5): Sigmoid()
  (6): Linear(in_features=64, out_features=32, bias=True)
  (7): ReLU()
  (8): Linear(in_features=32, out_features=10, bias=True)
  (9): LogSoftmax(dim=1)
)


In [ ]:
device = torch.device("cuda:0")
model = model.to(device)

In [ ]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


In [ ]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        images = images.view(images.shape[0], -1)
        outputs = model(images)
        loss = criterion(outputs, labels)
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

        

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:117: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Epoch [1/5], Step [100/420], Loss: 2.2688
Epoch [1/5], Step [200/420], Loss: 2.0277
Epoch [1/5], Step [300/420], Loss: 1.5106
Epoch [1/5], Step [400/420], Loss: 1.2321
Epoch [2/5], Step [100/420], Loss: 1.0700
Epoch [2/5], Step [200/420], Loss: 1.0371
Epoch [2/5], Step [300/420], Loss: 0.9485
Epoch [2/5], Step [400/420], Loss: 0.8853
Epoch [3/5], Step [100/420], Loss: 0.8263
Epoch [3/5], Step [200/420], Loss: 0.8827
Epoch [3/5], Step [300/420], Loss: 0.8782
Epoch [3/5], Step [400/420], Loss: 0.7869
Epoch [4/5], Step [100/420], Loss: 0.6333
Epoch [4/5], Step [200/420], Loss: 0.6439
Epoch [4/5], Step [300/420], Loss: 0.5847
Epoch [4/5], Step [400/420], Loss: 0.5655
Epoch [5/5], Step [100/420], Loss: 0.5045
Epoch [5/5], Step [200/420], Loss: 0.4365
Epoch [5/5], Step [300/420], Loss: 0.6574
Epoch [5/5], Step [400/420], Loss: 0.5342


In [ ]:
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        images = images.view(images.shape[0], -1)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:117: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Test Accuracy of the model on the 10000 test images: 81.09444444444445 %


In [ ]:

from torch import nn, optim
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear1 = nn.Linear(784, 256)
        self.linear2 = nn.Linear(256, 128)
        self.linear3 = nn.Linear(128, 64)
        self.linear4 = nn.Linear(64, 32)
        self.linear5 = nn.Linear(32, 16)
        self.linear6 = nn.Linear(16, 10)

        
        # Define activation 
        self.sigmoid = nn.Sigmoid()
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax()
        self.softmax1 = nn.Softmax(dim=1)
        
    def forward(self, x):
        x = self.linear1(x)
        x = self.sigmoid(x)

        x = self.linear2(x)
        x = self.softmax(x)

        x = self.linear3(x)
        x = self.relu(x)

        x = self.linear4(x)
        x = self.relu(x)

        x = self.linear5(x)
        x = self.relu(x)

        x = self.linear6(x)
        x = self.softmax1(x)
        
        return x

In [ ]:
model = Network()
model=model.cuda()

In [ ]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        images = images.view(images.shape[0], -1)
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:117: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Epoch [1/5], Step [100/420], Loss: 0.5894
Epoch [1/5], Step [200/420], Loss: 0.4438
Epoch [1/5], Step [300/420], Loss: 0.5588
Epoch [1/5], Step [400/420], Loss: 0.3996
Epoch [2/5], Step [100/420], Loss: 0.3945
Epoch [2/5], Step [200/420], Loss: 0.4351
Epoch [2/5], Step [300/420], Loss: 0.5069
Epoch [2/5], Step [400/420], Loss: 0.4241
Epoch [3/5], Step [100/420], Loss: 0.3616
Epoch [3/5], Step [200/420], Loss: 0.5516
Epoch [3/5], Step [300/420], Loss: 0.2100
Epoch [3/5], Step [400/420], Loss: 0.6739
Epoch [4/5], Step [100/420], Loss: 0.2478
Epoch [4/5], Step [200/420], Loss: 0.4261
Epoch [4/5], Step [300/420], Loss: 0.2699
Epoch [4/5], Step [400/420], Loss: 0.2786
Epoch [5/5], Step [100/420], Loss: 0.3658
Epoch [5/5], Step [200/420], Loss: 0.3071
Epoch [5/5], Step [300/420], Loss: 0.3655
Epoch [5/5], Step [400/420], Loss: 0.2508


In [ ]:
# Test the model
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        images = images.view(images.shape[0], -1)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:117: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Test Accuracy of the model on the 10000 test images: 86.70555555555555 %
